In [1]:
import bt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bt.core import Algo

In [2]:
data = pd.read_csv('./sp500_adj_close.csv',header=0, skiprows=1)
data = data[1:]
data.columns = ['datetime'] + data.columns[1:].to_list()
data['datetime'] = pd.to_datetime(data['datetime'])
data = data.set_index('datetime')
# special
data.drop(pd.to_datetime('2017-09-04'),inplace=True)
# data.loc['2018-01-01':]
data.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
datetime,,,,,,,,,,,,,,,,,,,,,
2010-01-04,20.545469,4.496876,39.103363,6.583586,NaN,22.512976,8.74,20.074522,33.718582,37.090000,...,19.830650,46.349365,32.941868,14.571863,NaN,20.328438,55.501415,28.670000,11.726357,NaN
2010-01-05,20.322289,5.005957,38.870968,6.594968,NaN,22.352354,8.53,19.912334,33.926979,37.700001,...,19.580618,46.530319,32.550251,14.588748,NaN,20.258917,57.258369,28.620001,12.139812,NaN
2010-01-06,20.250088,4.798555,39.209885,6.490066,NaN,22.141001,8.40,20.022911,34.287651,37.619999,...,19.447794,46.932491,32.764709,14.453672,NaN,20.114088,57.239891,28.400000,13.195447,NaN
2010-01-07,20.223829,4.939965,39.200214,6.478067,NaN,21.785934,8.40,20.188793,34.255581,36.889999,...,19.252451,46.785034,33.193615,14.521203,NaN,20.108294,58.552982,27.690001,14.673337,NaN
2010-01-08,20.217272,4.845690,39.355148,6.521136,NaN,22.022644,8.23,20.292002,34.119335,36.689999,...,19.533739,46.597363,33.193615,14.470551,NaN,20.114088,57.323116,27.600000,14.435819,NaN


In [3]:
benchmark_data = pd.read_csv('./sp500_^SPGC_adj_close.csv',index_col=1)
benchmark_data = benchmark_data[['^GSPC']]
benchmark_data

,^GSPC
datatime,
2010-01-04,1132.989990
2010-01-05,1136.520020
2010-01-06,1137.140015
2010-01-07,1141.689941
2010-01-08,1144.979980
...,...
2021-02-01,3773.860107
2021-02-02,3826.310059
2021-02-03,3830.169922


In [4]:
ema_50 = data.ewm(span=50, adjust=True).mean()
ema_100 = data.ewm(span=100, adjust=True).mean()
ema_150 = data.ewm(span=150, adjust=True).mean()
ema_200 = data.ewm(span=200, adjust=True).mean()

# 1. The current stock price is above both the 150-day and 200-day moving average
r_1 = (data > ema_150) + (data >ema_200)
# 2. The 150-day moving average is above the 200-day moving average
r_2 = ema_150 > ema_200
# 3. The 200-day moving average line is trending up for at least 1 month

# 4. The 50-day moving average is above both the 150-day and 200-day moving averages
r_4 = (ema_50 > ema_150) + (ema_50 > ema_200)
# 5. The current stock price is trading above the 50-day moving average
r_5 = data > ema_50
# 6. The current stock price is at least 30% above it’s 52-week low
low_52week = data.rolling(52 * 5).min()
r_6 = data >= low_52week * 1.3
# 7. The current stock price is within at least 25% of its 52-week high
high_52week = data.rolling(52 * 5).max()
r_7 = (data >= high_52week * 0.75) + (data <= high_52week * 1.25)
# 8. The relative strength ranking (as reported in Investor’s Business Daily) is no less than 70. Preferably 

signal = r_1 & r_2 & r_4 & r_5 & r_6 & r_7
signal

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
datetime,,,,,,,,,,,,,,,,,,,,,
2010-01-04,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2010-01-05,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2010-01-06,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2010-01-07,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2010-01-08,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-01,True,False,False,True,False,True,True,True,False,False,...,False,False,False,False,False,False,True,True,True,False
2021-02-02,True,False,False,True,False,True,True,True,False,True,...,False,False,True,False,False,True,True,True,True,False
2021-02-03,True,False,False,True,True,True,True,True,False,True,...,False,False,True,False,False,False,True,True,True,False


In [16]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns


class WeighEqually_DIY(Algo):

    def __init__(self,optimial_type='min_volatility',risk_free_rate = 0.1):
        super(WeighEqually_DIY, self).__init__()
        self.optimial_type = optimial_type
        self.risk_free_rate = risk_free_rate

    def __call__(self, target):
        selected = target.temp['selected']
        n = len(selected)

        if n == 0:
            target.temp['weights'] = {}
        else:
            
            df = target.universe[selected]
            # Calculate expected returns and sample covariance
            mu = expected_returns.mean_historical_return(df)
            S = risk_models.sample_cov(df)
            
            # Optimise for maximal Sharpe ratio
            ef = EfficientFrontier(mu, S)
            
            if self.optimial_type == 'min_volatility':
                
                weights = ef.min_volatility()
                cleaned_weights = ef.clean_weights()
                target.temp['selected'] = list(cleaned_weights.keys())
                target.temp['weights'] = cleaned_weights
            elif self.optimial_type == 'max_sharpe':
                weights = ef.max_sharpe(risk_free_rate = self.risk_free_rate)

                # risk_free_rate (float, optional) – risk-free rate of borrowing/lending, defaults to 0.02. The period of the risk-free rate should correspond to the frequency of expected returns.

                cleaned_weights = ef.clean_weights()
                target.temp['selected'] = list(cleaned_weights.keys())
                target.temp['weights'] = cleaned_weights
        return True


In [8]:
# create the strategy #risk
s_mv_monthly = bt.Strategy('minimum volatility monthly', [bt.algos.RunMonthly(),
                       bt.algos.SelectWhere(signal),
                       WeighEqually_DIY(optimial_type='min_volatility'),
                       bt.algos.Rebalance()])
# create a backtest and run it
mv_monthly = bt.Backtest(s_mv_monthly, data,progress_bar=True)

# create the strategy. #return(max risk)
s_msr_monthly = bt.Strategy('maximum sharpe ratio monthly', [bt.algos.RunMonthly(),
                       bt.algos.SelectWhere(signal),
                       WeighEqually_DIY(optimial_type='max_sharpe',risk_free_rate = 0.05),
                       bt.algos.Rebalance()])
# create a backtest and run it
msr__monthly = bt.Backtest(s_msr_monthly, data,progress_bar=True)

In [9]:
# create the strategy #risk
s_mv_yearly = bt.Strategy('minimum volatility yearly', [bt.algos.RunYearly(),
                       bt.algos.SelectWhere(signal),
                       WeighEqually_DIY(optimial_type='min_volatility'),
                       bt.algos.Rebalance()])
# create a backtest and run it
mv_yearly = bt.Backtest(s_mv_yearly, data,progress_bar=True)

# create the strategy. #return(max risk)
s_msr_yearly = bt.Strategy('maximum sharpe ratio yearly', [bt.algos.RunYearly(),
                       bt.algos.SelectWhere(signal),
                       WeighEqually_DIY(optimial_type='max_sharpe',risk_free_rate = 0.1),
                       bt.algos.Rebalance()])
# create a backtest and run it
msr__yearly = bt.Backtest(s_msr_yearly, data,progress_bar=True)

In [11]:
# # create the strategy benchmark 
# benchmark_yearly =  bt.Strategy('benchmark', [bt.algos.RunYearly(),
#                        bt.algos.SelectAll(),
#                        bt.algos.WeighEqually(),
#                        bt.algos.Rebalance()])
# # create a backtest and run it
# b_yearly = bt.Backtest(benchmark_yearly, benchmark_data, progress_bar=True)



In [17]:
res = bt.run(mv_monthly, msr__monthly,mv_yearly,msr__yearly)  #,b_yearly)

maximum sharpe ratio monthly
0% [##                            ] 100% | ETA: 00:00:00

Exception: Cannot allocate capital to A weight because price is 0 as of 2011-02-01 00:00:00

In [11]:
#AT the minimium risk the return is low and the maximum risk the return is high.

In [12]:
res.display()

Stat                 minimum volatility monthly    maximum sharpe ratio monthly    minimum volatility yearly    maximum sharpe ratio yearly    benchmark
-------------------  ----------------------------  ------------------------------  ---------------------------  -----------------------------  -----------
Start                2010-01-03                    2010-01-03                      2010-01-03                   2010-01-03                     2010-01-03
End                  2021-02-04                    2021-02-04                      2021-02-04                   2021-02-04                     2021-02-04
Risk-free rate       0.00%                         0.00%                           0.00%                        0.00%                          0.00%

Total Return         219.67%                       430.28%                         154.57%                      463.01%                        238.94%
Daily Sharpe         0.83                          0.90                            

In [13]:
res.plot().get_figure().savefig("output_res_plot.png")

In [14]:
res.prices[:]

,minimum volatility monthly,maximum sharpe ratio monthly,minimum volatility yearly,maximum sharpe ratio yearly,benchmark
2010-01-03,100.000000,100.000000,100.000000,100.000000,100.000000
2010-01-04,100.000000,100.000000,100.000000,100.000000,100.000000
2010-01-05,100.000000,100.000000,100.000000,100.000000,100.311349
2010-01-06,100.000000,100.000000,100.000000,100.000000,100.366032
2010-01-07,100.000000,100.000000,100.000000,100.000000,100.767336
...,...,...,...,...,...
2021-01-29,324.506310,512.085782,252.352670,536.005366,327.666250
2021-02-01,325.706713,525.420283,253.286008,553.843201,332.924744
2021-02-02,323.001966,534.661859,255.371959,568.586680,337.550830
2021-02-03,322.109241,526.495815,254.418271,560.135499,337.891270


In [43]:
bubble = res.display_monthly_returns()
type(bubble)

  Year    Jan    Feb    Mar    Apr    May    Jun    Jul    Aug    Sep    Oct    Nov    Dec    YTD
------  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----
  2010   0      0      0      0      0      0      0      0      0      0      0      0      0
  2011   0      6.29   4.46   5.49   1.47  -1.06  -2.11  -1.92  -0.98   2.01   1.65   2.65  19.01
  2012  -1.43   1.33   2.48   3.15  -0.93   4.78   1.92  -3.56   1.63  -2.11  -1.03  -0.58   5.44
  2013   4.6    4.59   5.12   2.56  -3.34   0.68   6.42  -4.63   5.42   5.34  -0.16  -0.64  28.35
  2014  -0.45   3.3   -0.17   0.92   0.19   1.54  -5.21   3.57  -0.35   4.82   4.82  -0.13  13.16
  2015   1.74  -0.5   -0.84  -4.07   1.58   1.4    3.8   -3.15   0.48   2.91  -0.77   1.28   3.62
  2016  -0.1    1.66   5.03  -0.7    0.2    3.61   0.83  -0.9   -0.01  -2.68  -0.4   -1.83   4.55
  2017   0.62   3.09   0.56   2.54   3.32  -0.69   1.24   0.51   1.31   4.25   4.91  -0.84  22.71
  2018   4.96  -5.06   

NoneType

,ABC,AWK,AZO,CHTR,COO,COST,DVA,EW,FIS,HRL,...,CTXS,EQIX,NLOK,WST,LIN,CERN,GOOG,PSA,AON,FLIR
2010-01-03,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
2010-01-04,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
2010-01-05,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
2010-01-06,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
2010-01-07,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-29,0.0,0.0,0.07582,0.018348,0.0,0.037577,0.008246,0.0,0.0,0.0,...,0.0,0.0,0.024829,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
2021-02-01,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.014409,0.0,0.0,0.0,...,0.0,0.0,0.030319,0.0,0.0,0.0,0.0,0.021218,0.007488,0.000754
2021-02-02,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.013652,0.0,0.0,0.0,...,0.0,0.0,0.031172,0.0,0.0,0.0,0.0,0.021176,0.007678,0.000772
2021-02-03,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.013549,0.0,0.0,0.0,...,0.0,0.0,0.030930,0.0,0.0,0.0,0.0,0.021327,0.007520,0.000773


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
 
 
# Change color with c and alpha
plt.scatter(x, y, s=z*4000, c="red", alpha=0.4)
#plt.show()


In [35]:
res.display_monthly_returns(1)

  Year    Jan    Feb     Mar    Apr    May    Jun    Jul    Aug    Sep    Oct    Nov     Dec    YTD
------  -----  -----  ------  -----  -----  -----  -----  -----  -----  -----  -----  ------  -----
  2010   0      0       0      0      0      0      0      0      0      0      0       0      0
  2011   0      0.15    4.46   3.36   3.38   0.54  -4.75  -0.96   1.23   3.49   1.08    1.76  14.26
  2012   2.93   8.1     4.93   2.78  -1.31   1.84  -1.21   1.01   1.14  -4.4    2.21    0.48  19.51
  2013   4.8    5.11    7.09   3.17   7.71  -4.24   6.36  -0.14   9.18   0.71   0.21    1.89  49.69
  2014  -1.03  11.84    0.55   2.39   3.34   7.65  -5.02   5.79  -6.29   4.19   6.91   -0.46  32.37
  2015  -1.41   4.76    0.9    0.92   1.42  -0.96   2.57  -3     -0.73   6.8    0.5    -3.23   8.38
  2016  -0.03  -1.66    3.21  -3.27   2.94   3.51   2.08   0.19   0.96   1.02  -1.16   -4.31   3.18
  2017   4.14   6.59   -1.21   0.92   7.32   0.81  -3.96   0.69   2.43   1.65   2.52    0.87  24.67
  2

In [36]:
res.display_monthly_returns(2)

  Year    Jan    Feb     Mar    Apr    May    Jun    Jul    Aug    Sep    Oct    Nov    Dec    YTD
------  -----  -----  ------  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----
  2010   0      0       0      0      0      0      0      0      0      0      0      0      0
  2011   0      0       0      0      0      0      0      0      0      0      0      0      0
  2012  -0.44   2.06    2.52   3.25  -2.89   3.85   0.49  -1.6    0.63  -3.71   2.96  -1.57   5.33
  2013   3.76   4.56    6.18   0.26  -1.96  -0.8    5.85  -3.16   1      3.72   0.12   0.36  21.18
  2014  -0.4    7.33   -1.08  -1.91   1.56   0.8   -3.68   4.3   -1.29   5.59   4.67   1.9   18.61
  2015   1.76   1.05    0.08  -1.4    2.62  -2.36   6.37  -1.97  -0.05   2.56  -1.17   1.56   9.09
  2016   0.25  -1.02    4.94  -3.61  -2.32   1.71  -0.97  -1.15  -0.48  -1      1.19   2.38  -0.38
  2017  -0.39   2.02   -0.18   0.06   1.79  -1.22   3.27  -0.39   0.44   0.27   3.45   1.83  11.39
  2018   5.52  -

In [37]:
res.display_monthly_returns(3)

  Year    Jan    Feb     Mar    Apr    May    Jun    Jul    Aug    Sep     Oct    Nov    Dec    YTD
------  -----  -----  ------  -----  -----  -----  -----  -----  -----  ------  -----  -----  -----
  2010   0      0       0      0      0      0      0      0      0       0      0      0      0
  2011   0      0       0      0      0      0      0      0      0       0      0      0      0
  2012   4.34   9.01    6.59   2.09  -4.14   1.39  -4.15  -0.01   5.47  -12.75   4.2    2.46  13.26
  2013   3.72   2.99    6.72   4.2    8.73  -2.13   9.06  -3.64  11.03    0.16   2.12   1.46  53.07
  2014  -0.47  15.81   -0.4    2.52   3.83   8.39  -4.07   5.61  -8.49    0.22  -6.22   2.42  18.2
  2015  -0.87   6.5     0.93   2.26   1.04  -0.75   4.48  -1.18  -1.2     6.55  -1.83  -0.96  15.48
  2016   0.29  -3.42    8.45  -4.78   3.4    3.55   0.08  -1.71  -1.35   -2.39  -2.46   2.57   1.5
  2017   5.14   5.24    0.11   0.63   1.61  -1.1    4.37  -1.81   1.48    1.21   1.35   2.77  22.79
  2018  

In [39]:
res.display_monthly_returns(4)

  Year    Jan    Feb     Mar    Apr    May    Jun    Jul    Aug    Sep    Oct    Nov    Dec    YTD
------  -----  -----  ------  -----  -----  -----  -----  -----  -----  -----  -----  -----  -----
  2010  -5.21   2.85    5.88   1.47  -8.19  -5.38   6.87  -4.74   8.75   3.68  -0.23   6.53  10.99
  2011   2.26   3.19   -0.1    2.85  -1.35  -1.82  -2.15  -5.68  -7.17  10.76  -0.51   0.85  -0
  2012   4.36   4.06    3.13  -0.75  -6.26   3.95   1.26   1.98   2.42  -1.98   0.28   0.71  13.4
  2013   5.04   1.11    3.6    1.81   2.08  -1.5    4.94  -3.13   2.97   4.46   2.8    2.36  29.58
  2014  -3.56   4.31    0.69   0.62   2.1    1.91  -1.51   3.76  -1.55   2.32   2.45  -0.42  11.39
  2015  -3.1    5.49   -1.74   0.85   1.05  -2.1    1.97  -6.26  -2.64   8.29   0.05  -1.75  -0.73
  2016  -5.07  -0.41    6.6    0.27   1.53   0.09   3.56  -0.12  -0.12  -1.94   3.42   1.82   9.53
  2017   1.79   3.72   -0.04   0.91   1.16   0.48   1.93   0.05   1.93   2.22   2.81   0.98  19.41
  2018   5.62 

In [20]:
res.to_csv(path='./res_performanceStats.csv')

In [45]:
res.prices[:]

,minimum volatility monthly,maximum sharpe ratio monthly,minimum volatility yearly,maximum sharpe ratio yearly,benchmark
2010-01-03,100.000000,100.000000,100.000000,100.000000,100.000000
2010-01-04,100.000000,100.000000,100.000000,100.000000,100.000000
2010-01-05,100.000000,100.000000,100.000000,100.000000,100.311349
2010-01-06,100.000000,100.000000,100.000000,100.000000,100.366032
2010-01-07,100.000000,100.000000,100.000000,100.000000,100.767336
...,...,...,...,...,...
2021-01-29,324.506310,512.085782,252.352670,536.005366,327.666250
2021-02-01,325.706713,525.420283,253.286008,553.843201,332.924744
2021-02-02,323.001966,534.661859,255.371959,568.586680,337.550830
2021-02-03,322.109241,526.495815,254.418271,560.135499,337.891270


In [22]:
fig, axs = plt.subplots()
sns.lineplot(data=res.prices).get_figure().savefig('output.png')


In [23]:
df = res.prices[:]
df = df/100 -1
df

,minimum volatility monthly,maximum sharpe ratio monthly,minimum volatility yearly,maximum sharpe ratio yearly,benchmark
2010-01-03,0.000000,0.000000,0.000000,0.000000,0.000000
2010-01-04,0.000000,0.000000,0.000000,0.000000,0.000000
2010-01-05,0.000000,0.000000,0.000000,0.000000,0.003113
2010-01-06,0.000000,0.000000,0.000000,0.000000,0.003660
2010-01-07,0.000000,0.000000,0.000000,0.000000,0.007673
...,...,...,...,...,...
2021-01-29,2.245063,4.120858,1.523527,4.360054,2.276662
2021-02-01,2.257067,4.254203,1.532860,4.538432,2.329247
2021-02-02,2.230020,4.346619,1.553720,4.685867,2.375508
2021-02-03,2.221092,4.264958,1.544183,4.601355,2.378913


In [27]:
daily_return = df.shift(1) - df
daily_return.tail()

,minimum volatility monthly,maximum sharpe ratio monthly,minimum volatility yearly,maximum sharpe ratio yearly,benchmark
2021-01-29,0.067096,0.121712,0.052155,0.129658,0.064509
2021-02-01,-0.012004,-0.133345,-0.009333,-0.178378,-0.052585
2021-02-02,0.027047,-0.092416,-0.020860,-0.147435,-0.046261
2021-02-03,0.008927,0.081660,0.009537,0.084512,-0.003404
2021-02-04,0.024368,-0.037800,-0.001506,-0.028766,-0.010522


In [30]:
daily_return.plot(figsize=(16,9)).get_figure().savefig('output_daily_return.png')

In [33]:
daily_return[['minimum volatility monthly','benchmark']].plot(figsize=(16,9)).get_figure().savefig('output_daily_return_benchmark_minimum_volatility_monthly.png')

In [34]:
daily_return[['maximum sharpe ratio monthly','benchmark']].plot(figsize=(16,9)).get_figure().savefig('output_daily_return_benchmark_maximum_sharpe_ratio_monthly.png')

In [31]:
daily_return.mean()

minimum volatility monthly     -0.000787
maximum sharpe ratio monthly   -0.001541
minimum volatility yearly      -0.000554
maximum sharpe ratio yearly    -0.001658
benchmark                      -0.000856
dtype: float64

In [56]:
df[:].plot(figsize=(16,9)).get_figure().savefig('output_benchmark.png')

In [42]:
# Rolling Volatility
(res.prices.pct_change().rolling(window=12*20).std()*np.sqrt(252)).plot().get_figure().savefig("Rolling Volatility.png")

[]

In [50]:
res.get_security_weights().plot().get_figure().savefig("./security_weights.png")

In [53]:
res.get_security_weights().to_csv('./security_weights.csv')

In [61]:
res.get_transactions().to_csv('./transactions.csv')

In [132]:
transactions_min_v_monthly = res.get_transactions('minimum volatility monthly')
transactions_min_v_monthly.to_csv('./transactions_min_v_monthly.csv')
transactions_min_v_monthly

price  quantity
Date       Security                      
2011-02-01 ABC        31.034409     170.0
           AWK        20.379272     487.0
           AZO       255.000000     523.0
           CHTR       41.639999    4607.0
           COO        57.254578     457.0
...                         ...       ...
2021-02-01 TGT       183.570007     806.0
           TWTR       52.660000     146.0
           VRSK      185.940002    -258.0
           WLTW      207.449997     784.0
           WMT       139.270004   -3484.0

[3216 rows x 2 columns]

In [133]:
transactions_max_sr_monthly = res.get_transactions('maximum sharpe ratio monthly')
transactions_max_sr_monthly.to_csv('./transactions_max_sr_monthly.csv')
transactions_max_sr_monthly

price  quantity
Date       Security                      
2011-02-01 ALXN       42.825001     799.0
           AZO       255.000000     650.0
           CHTR       41.639999     533.0
           EW         14.323333   14164.0
           FTNT       19.730000    5323.0
...                         ...       ...
2021-02-01 POOL      359.820007    -352.0
           STZ       215.059998    -323.0
           TDG       570.309998     -65.0
           TSLA      839.809998     -41.0
           WST       297.019989     245.0

[1840 rows x 2 columns]

In [134]:
transactions_min_v_yearly = res.get_transactions('minimum volatility yearly')
transactions_min_v_yearly.to_csv('./transactions_min_v_yearly.csv')
transactions_min_v_yearly

price  quantity
Date       Security                      
2012-01-03 AAP        67.470627     483.0
           AMGN       51.639805     579.0
           BMY        26.591764    7131.0
           CHD        19.679581    3181.0
           CHTR       57.189999      91.0
...                         ...       ...
2021-01-04 T          28.928171   -3254.0
           TGT       177.630005     105.0
           VRSK      202.880005      31.0
           WLTW      203.699997     -38.0
           WMT       146.529999    1550.0

[299 rows x 2 columns]

In [135]:
transactions_max_sr_yearly = res.get_transactions('maximum sharpe ratio yearly')
transactions_max_sr_yearly.to_csv('./transactions_max_sr_yearly.csv')
transactions_max_sr_yearly

price  quantity
Date       Security                      
2012-01-03 ALXN       70.570000    3004.0
           CMG       341.269989     788.0
           DLTR       41.215000    6697.0
           DPZ        28.185535    8617.0
2013-01-02 ALXN      100.050003   -1599.0
...                         ...       ...
2021-01-04 NFLX      522.859985     905.0
           PAYC      427.730011    1208.0
           POOL      356.890015    1150.0
           TDG       587.669983    -348.0
           TSLA      729.770020    1089.0

[147 rows x 2 columns]

In [136]:
transactions_benchmark = res.get_transactions('benchmark')
transactions_benchmark.to_csv('./transactions_benchmark.csv')
transactions_benchmark

,,price,quantity
Date,Security,,
2010-01-04,Adj Close,1132.98999,882.0


In [128]:
(transactions.loc['2011-02-01']['price'] * transactions.loc['2011-02-01']['quantity']).sum()

999405.0350484848

In [77]:
transactions.index

MultiIndex([('2011-02-01',  'ABC'),
            ('2011-02-01',  'AWK'),
            ('2011-02-01',  'AZO'),
            ('2011-02-01', 'CHTR'),
            ('2011-02-01',  'COO'),
            ('2011-02-01', 'COST'),
            ('2011-02-01',  'DVA'),
            ('2011-02-01',   'EW'),
            ('2011-02-01',  'FIS'),
            ('2011-02-01',  'HRL'),
            ...
            ('2021-02-01', 'NFLX'),
            ('2021-02-01', 'NLOK'),
            ('2021-02-01',  'NVR'),
            ('2021-02-01',  'PPL'),
            ('2021-02-01',  'PSA'),
            ('2021-02-01',  'TGT'),
            ('2021-02-01', 'TWTR'),
            ('2021-02-01', 'VRSK'),
            ('2021-02-01', 'WLTW'),
            ('2021-02-01',  'WMT')],
           names=['Date', 'Security'], length=3216)

In [123]:
stocks_lst = []
for i in transactions.index:
    stocks_lst.append(i[1])
stocks_lst = list(set(stocks))
stocks_lst[:5]

['LUMN', 'DRI', 'FE', 'BMY', 'APTV']

In [120]:
lst = []
for name in stocks_lst:
    stock = transactions.loc[(slice(None),name),:]
    stocks_held = stock['quantity'].sum()
    profit = (stock['price']*stock['quantity']).sum()
    nums_trade = stock.shape[0]
    lst.append([stocks_held, profit, nums_trade])

df_summary = pd.DataFrame(data = np.array(lst), index= stocks_lst, columns=['hold', 'profit', 'nums of trade'])

df_summary

,hold,profit,nums of trade
LUMN,0.0,880.427952,15.0
DRI,0.0,-1328.600815,5.0
FE,0.0,2652.262096,16.0
BMY,0.0,-21733.123657,49.0
APTV,0.0,-34344.150965,23.0
...,...,...,...
ECL,0.0,-782.780533,2.0
BIIB,0.0,1523.560150,2.0
MO,0.0,-100419.681570,35.0
GILD,0.0,412.908543,6.0


In [129]:
df_summary.to_csv('./transactions_summary.csv')

In [121]:
df_summary.sort_values("profit",inplace=False)

,hold,profit,nums of trade
AZO,0.0,-158975.778229,71.0
MCD,0.0,-148278.865112,35.0
PG,0.0,-142300.256634,34.0
WST,0.0,-137624.534668,2.0
NEM,0.0,-126682.639452,36.0
...,...,...,...
TGT,1036.0,168504.450340,43.0
BDX,695.0,216584.690422,31.0
DUK,5242.0,487718.798828,10.0
JNJ,4255.0,676258.383972,23.0


In [122]:
df_summary['profit'].sum()

996282.687652589